# Use scikit-learn and custom library to predict temperature with `ibm-watson-machine-learning`

This notebook contains steps and code to train a Scikit-Learn model that uses a custom defined transformer and use it with Watson Machine Learning service. Once the model is trained, this notebook contains steps to persist the model and custom defined transformer to Watson Machine Learning Repository, deploy and score it using Watson Machine Learning python client.

In this notebook, we use GNFUV dataset that contains mobile sensor readings data about humidity and temperature from Unmanned Surface Vehicles in a test-bed in Athens, to train a Scikit-Learn model for predicting the temperature. 

Some familiarity with Python is helpful. This notebook uses Python & scikit-learn.

## Learning goals

The learning goals of this notebook are:

- Train a model with custom defined transformer
- Persist the custom defined transformer and the model in Watson Machine Learning repository.
- Deploy the model using Watson Machine Learning Service
- Perform predictions using the deployed model

## Contents
1. [Set up the environment](#setup)
2. [Install python library containing custom transformer implementation](#install_lib)
3. [Prepare training data](#load)
4. [Train the scikit-learn model](#train)
5. [Save the model and library to WML Repository](#upload)
6. [Deploy and score data in the IBM Cloud](#deploy)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)


<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/admin/create-services.html?context=cpdaas&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).


### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.


In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

<a id="install_lib"></a>

## 2. Install the library containing custom transformer

Library - `linalgnorm-0.1` is a python distributable package that contains the implementation of a user defined Scikit-Learn transformer - `LNormalizer` . <br>
Any 3rd party libraries that are required for the custom transformer must be defined as the dependency for the corresponding library that contains implementation of the transformer. 


In this section, we will create the library and install it in the current notebook environment. 

In [ ]:
!mkdir -p linalgnorm-0.1/linalg_norm

Define a custom scikit transformer.

In [ ]:
%%writefile linalgnorm-0.1/linalg_norm/sklearn_transformers.py

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


class LNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, norm_ord=2):
        self.norm_ord = norm_ord
        self.row_norm_vals = None

    def fit(self, X, y=None):
        self.row_norm_vals = np.linalg.norm(X, ord=self.norm_ord, axis=0)

    def transform(self, X, y=None):
        return X / self.row_norm_vals

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

    def get_norm_vals(self):
        return self.row_norm_vals

Wrap created code into Python source distribution package.

In [ ]:
%%writefile linalgnorm-0.1/linalg_norm/__init__.py

__version__ = "0.1"

In [ ]:
%%writefile linalgnorm-0.1/README.md

A simple library containing a simple custom scikit estimator.

In [ ]:
%%writefile linalgnorm-0.1/setup.py

from setuptools import setup

VERSION='0.1'
setup(name='linalgnorm',
      version=VERSION,
      url='https://github.ibm.com/NGP-TWC/repository/',
      author='IBM',
      author_email='ibm@ibm.com',
      license='IBM',
      packages=[
            'linalg_norm'
      ],
      zip_safe=False
)

In [ ]:
%%bash

cd linalgnorm-0.1
python setup.py sdist --formats=zip
cd ..
mv linalgnorm-0.1/dist/linalgnorm-0.1.zip .
rm -rf linalgnorm-0.1

Install the downloaded library using `pip` command

In [ ]:
!pip install linalgnorm-0.1.zip

<a id="load"></a>

## 3. Download training dataset and prepare training data

Download the data from UCI repository - https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip

In [ ]:
!rm -rf dataset
!mkdir dataset

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip --output-document=dataset/gnfuv_dataset.zip

In [ ]:
!unzip dataset/gnfuv_dataset.zip -d dataset

Create pandas datafame based on the downloaded dataset

In [ ]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from json import JSONDecodeError

In [ ]:
home_dir = './dataset'
pi_dirs = os.listdir(home_dir)

data_list = []
base_time = None
columns = None

for pi_dir in pi_dirs:
    if 'pi' not in pi_dir:
        continue
    curr_dir = os.path.join(home_dir, pi_dir)
    data_file = os.path.join(curr_dir, os.listdir(curr_dir)[0])
    with open(data_file, 'r') as f:
        line = f.readline().strip().replace("'", '"')
        while line != '':
            try:
                input_json = json.loads(line)
                sensor_datetime = datetime.fromtimestamp(input_json['time'])
                if base_time is None:
                    base_time = datetime(sensor_datetime.year, sensor_datetime.month, sensor_datetime.day, 0, 0, 0, 0)
                input_json['time'] = (sensor_datetime - base_time).seconds
                data_list.append(list(input_json.values()))
                if columns is None:
                    columns = list(input_json.keys())
            except JSONDecodeError as je:
                pass
            line = f.readline().strip().replace("'", '"')

data_df = pd.DataFrame(data_list, columns=columns)

In [ ]:
data_df.head()

Create training and test datasets from the downloaded GNFUV-USV dataset.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Y = data_df['temperature']
X = data_df.drop('temperature', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=143)

<a id="train"></a>

## 4. Train a model

In this section, you will use the custom transformer as a stage in the Scikit-Learn `Pipeline` and train a model.

#### Import the custom transformer 
Here, import the custom transformer that has been defined in `linalgnorm-0.1.zip` and create an instance of it that will inturn be used as stage in `sklearn.Pipeline`

In [ ]:
from linalg_norm.sklearn_transformers import LNormalizer

In [ ]:
lnorm_transf = LNormalizer()

Import other objects required to train a model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

Now, you can create a `Pipeline` with user defined transformer as one of the stages and train the model

In [ ]:
skl_pipeline = Pipeline(steps=[('normalizer', lnorm_transf), ('regression_estimator', LinearRegression())])
skl_pipeline.fit(X_train.loc[:, ['time', 'humidity']].values, y_train)

In [ ]:
y_pred = skl_pipeline.predict(X_test.loc[:, ['time', 'humidity']].values)
rmse = np.mean((np.round(y_pred) - y_test.values)**2)**0.5
print('RMSE: {}'.format(rmse))

<a id="upload"></a>

## 5. Persist the model and custom library

In this section, using `ibm-watson_machine_learning` SDK, you will ...
- save the library `linalgnorm-0.1.zip` in WML Repository by creating a package extension resource
- create a Software Specification resource and bind the package resource to it. This Software Specification resource will be used to configure the online deployment runtime environment for a model 
- bind Software Specification resource to the model and save the model to WML Repository

### Create package extension

Define the meta data required to create package extension resource. <br>

The value for `file_path` in `client.package_extensions.LibraryMetaNames.store()` contains the library file name that must be uploaded to the WML.

**Note:** You can also use conda environment configuration file `yaml` as package extension input. In such case set the `TYPE` to `conda_yml` and `file_path` to yaml file.
```
client.package_extensions.ConfigurationMetaNames.TYPE = "conda_yml"
```

In [ ]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "K_Linag_norm_skl",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Pkg extension for custom lib",
    client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}

pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="linalgnorm-0.1.zip")
pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)


Display the details of the package extension resource that was created in the above cell.

In [ ]:
details = client.package_extensions.get_details(pkg_extn_uid)

### Create software specification and add custom library

Define the meta data required to create software spec resource and bind the package. This software spec resource will be used to configure the online deployment runtime environment for a model.

In [ ]:
client.software_specifications.ConfigurationMetaNames.show()

#### List base software specifications

In [ ]:
client.software_specifications.list()

#### Select base software specification to extend

In [ ]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("runtime-23.1-py3.10")

#### Define new software specification based on base one and custom library

In [ ]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)


client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

### Save the model

Define the metadata to save the trained model to WML Repository along with the information about the software spec resource required for the model. 

The `client.repository.ModelMetaNames.SOFTWARE_SPEC_UID` metadata property is used to specify the GUID of the software spec resource that needs to be associated with the model.

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Temp prediction model with custom lib",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
    
}

Save the model to the WML Repository and display its saved metadata. 

In [ ]:
published_model = client.repository.store_model(model=skl_pipeline, meta_props=model_props)

In [ ]:
published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

<a id="deploy"></a>

## 6 Deploy and Score

In this section, you will deploy the saved model that uses the custom transformer and perform predictions. You will use WML client to perform these tasks.

### Deploy the model

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of custom lib model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

### Predict using the deployed model

**Note**: Here we use deployment `uid` saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.

In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

<a id="score"></a>
Prepare the payload for prediction. The payload contains the input records for which predictions has to be performed.

In [ ]:
scoring_payload = {
    "input_data": [{
        'fields': ["time", "humidity"],
        'values': [[79863, 47]]}]
}

Execute the method to perform online predictions and display the prediction results

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>

## 8. Summary

You successfully completed this notebook! 
 
You learned how to use a scikit-learn model with custom transformer in Watson Machine Learning service to deploy and score.

Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

## Author

**Krishnamurthy Arthanarisamy**, is a senior technical lead in IBM Watson Machine Learning team. Krishna works on developing cloud services that caters to different stages of machine learning and deep learning modeling life cycle.

**Lukasz Cmielowski**, PhD, is a Software Architect and Data Scientist at IBM.

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.